# Notebook 3: json parsing

## Fields

- `hosts`: list of strings, defined from env like `["foo", "bar"]`
- `ports`: definition of another model, which input must be a JSON compliant with the model:
    - `http`: int (required)
    - `mqtt`: int (required)
    - ...defined from env like `{"http": 80, "mqtt": 1883}` - the whole `ports` object is optional, but if defined, both of its fields are required

In [1]:
!cat 3*.py

from typing import List, Optional

from pydantic import BaseSettings, BaseModel


class PortsSettings(BaseModel):
	http: int
	mqtt: int


class MySettings(BaseSettings):
    hosts: List[str]
    ports: Optional[PortsSettings]
    
    class Config:
        env_prefix = "APP_"
        env_file = ".env"


settings = MySettings()
print("App hosts:", settings.hosts)
if settings.ports:
	print("App HTTP port:", settings.ports.http)
	print("App MQTT port:", settings.ports.mqtt)


In [2]:
# With valid hosts array (valid JSON string)
!APP_HOSTS="[\"foo\", \"bar\"]" python 3*.py

App hosts: ['foo', 'bar']


In [3]:
# With hosts array, including a number that is converted to string
!APP_HOSTS="[\"foo\", 1]" python 3*.py

App hosts: ['foo', '1']


In [4]:
# With hosts array, including an invalid object on the array
!APP_HOSTS="[\"foo\", {}]" python 3*.py

Traceback (most recent call last):
  File "3-json_parsing.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 338, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for MySettings
hosts -> 1
  str type expected (type=type_error.str)


In [5]:
# Invalid JSON array of Hosts
!APP_HOSTS="this is a string, not valid json!!!" python 3*.py

Traceback (most recent call last):
  File "pydantic/env_settings.py", line 62, in pydantic.env_settings.BaseSettings._build_environ
  File "/home/david/.miniconda3/envs/jupyter/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/home/david/.miniconda3/envs/jupyter/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/david/.miniconda3/envs/jupyter/lib/python3.8/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "3-json_parsing.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/env_settings.py", line 31, in pydantic.env_settin

In [6]:
# Set hosts array to the .env file, for the next examples working with the ports object
!echo "APP_HOSTS=[\"hosts\", \"in\", \"the\", \".env\", \"file\"]" > .env

In [7]:
# Valid ports object (JSON)
!APP_PORTS="{\"http\": 8080, \"mqtt\": 1881}" python 3*.py

App hosts: ['hosts', 'in', 'the', '.env', 'file']
App HTTP port: 8080
App MQTT port: 1881


In [8]:
# Ports defined as string or float are converted to int
!APP_PORTS="{\"http\": \"8080\", \"mqtt\": 1883.95}" python 3*.py

App hosts: ['hosts', 'in', 'the', '.env', 'file']
App HTTP port: 8080
App MQTT port: 1883


In [9]:
# Unparseable ports
!APP_PORTS="{\"http\": {}, \"mqtt\": \"this is a string\"}" python 3*.py

Traceback (most recent call last):
  File "3-json_parsing.py", line 20, in <module>
    settings = MySettings()
  File "pydantic/env_settings.py", line 28, in pydantic.env_settings.BaseSettings.__init__
  File "pydantic/main.py", line 338, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 2 validation errors for MySettings
ports -> http
  value is not a valid integer (type=type_error.integer)
ports -> mqtt
  value is not a valid integer (type=type_error.integer)
